# Notebook explaining the nuts and bots of data cartography implementation in enginora

Implementation of the concept of data cartography as outlined in the paper: https://aclanthology.org/2020.emnlp-main.746/. As the authors write:

 *Large datasets have become commonplace in NLP research. However, the increased emphasis on data quantity has made it challenging to assess the quality of data. We introduce Data Maps—a model-based tool to characterize and diagnose datasets. We leverage a largely ignored source of information: thebehavior of the model on individual instances during training (training dynamics) for building data maps. This yields two intuitive measures for each example—the model’s confidence in the true class, and the variability ofthis confidence across epochs—obtained in a single run of training. Experiments across four datasets show that these model-dependent measures reveal three distinct regions in the data map, each with pronounced characteristics. First, our data maps show the presence of ambiguous regions with respect to the model, which contribute the most towards out-of-distribution generalization. Second, the most populous regions in the data are easy to learn for the model, and play an important role in model optimization. Finally, data maps uncover a region with instances that the model finds hard to learn; these often correspond to labeling errors. Our results indicate that a shift in focus from quantity to quality of data could lead to robust models and improved out-of-distribution generalization.*

## Step 1: 
Run Data Cartography, in the training config, set perform_data_cartography to True, and make sure you provide a distinict validatoon set, so that data cartography can be conducted on the whole of the training set. You can set a file destination for the data cartography results.
Make sure it looks something like this:

```
training:
  dataset_path: ../../data/featuresets/thedeep.subset.train.txt
  validation_dataset_path: ../../data/featuresets/thedeep.subset.validation.txt
  selectors:
    - args: { }
      name: dummy
  batch_size: 2
  epochs: 2
  learning_rate: 1e-3
  validation_size: 0.2
  output_dir: ClassificationBERT # set data_seed
  metric_for_best_model: f1_score
  perform_data_cartography: True
```

In [1]:
import enginora
import logging
import os

In [2]:
os.environ['WANDB_DISABLED'] = 'true'

logging.basicConfig(format="[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s", level=logging.INFO)

In [ ]:
enginora.loop(config_path="./config_run_data_cartography.yaml")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss


## Step 2:Perform experiments on subsets. 

You can now perform experiments: sampling data directly from the most ambiguous examples, the hard-to-learn examples and the easy-to-learn examples, You can also add randomly sapled fraction of data. The data will not allow for sampling with replacement.


Ensure now your config has perform_data_cartography set to False, and the data cartography selector is enabled!

```

training:
  dataset_path: ../../data/featuresets/thedeep.subset.train.txt
  validation_dataset_path: ../../data/featuresets/thedeep.subset.validation.txt
  selectors:
    - args: {
              hard_to_learn: 0,
              easy-to-learn: 0.3,
              ambiguous: 0.1,
              random: 0.6

    }
      name: data_cartography
  batch_size: 2
  epochs: 2
  learning_rate: 1e-3
  validation_size: 0.2
  output_dir: ClassificationBERT # set data_seed
  metric_for_best_model: f1_score

testing:
  dataset_path: ../../data/featuresets/thedeep.subset.test.txt
  selectors: [ ]
  results_file: ./file_test_results_experiment.pickle

control:
  dataset_path: ../../data/featuresets/thedeep.subset.control.txt
  results_file: ./file_control_results_experiment.pickle
```

In [7]:
enginora.loop(config_path="./config_experiments_from_data_cartography_results.yaml")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\ismyn/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve

EASY INDCICES: 9 Int64Index([16, 6, 13], dtype='int64')
HARD INDCICES: 0 []
AMBIGUOUS INDCICES: 3 Int64Index([0, 9, 27, 25, 18, 16, 14, 13, 10], dtype='int64')
RANDOM INDCICES: Int64Index([0, 5, 20], dtype='int64')
indices: [0, 5, 6, 9, 10, 13, 14, 16, 18, 20, 25, 27]
DATASET:        id                                               text  label
0    5446  In addition to the immediate life-saving inter...      9
5    8318  Secretary-General Antonio Guterres says in a d...      9
6    3464  Safe access to basic services and adequate liv...      9
9   12607  In the refugee camps of Nord and Sud Ubangi, a...      4
10   1619  The Bidi Bidi settlement has sprung up at an e...      9
13   9421  The Interior Ministry’s regular police and its...      9
14   6027  The situation is especially bad for communitie...      4
16  15671  Security forces used unlawful force and threat...      9
18  10090  t the same time, too many girls marry and beco...      9
20   4620  More emergency supplies includi

Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Saving model checkpoint to ClassificationBERT_experiment\checkpoint-6
Configuration saved in ClassificationBERT_experiment\checkpoint-6\config.json
Model weights saved in ClassificationBERT_experiment\checkpoint-6\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Saving model checkpoint to ClassificationBERT_experiment\checkpoint-12
Configuration saved in ClassificationBERT_experiment\checkpoint-12\config.json
Model weights saved in ClassificationBERT_experiment\checkpoint-12\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Saving model checkpoint to ClassificationBERT_experiment\checkpoint-18
Configuration saved in ClassificationBERT_experiment\checkpoint-18\config.json
Model weights saved in ClassificationBERT_experiment\checkpoint-18\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Saving model checkpoint to

***** Running Prediction *****
  Num examples = 10
  Batch size = 2


{'train_results': {'train_runtime': 123.4428,
  'train_samples_per_second': 0.583,
  'train_steps_per_second': 0.292,
  'total_flos': 9472848445440.0,
  'train_loss': 1.4648439354366727,
  'epoch': 6.0},
 'test_results': {'test_loss': 3.715395450592041,
  'test_f1_score': 0.030303030303030304,
  'test_runtime': 3.7093,
  'test_samples_per_second': 2.696,
  'test_steps_per_second': 1.348},
 'control_results': {'test_loss': 5.031211853027344,
  'test_f1_score': 0.06666666666666668,
  'test_runtime': 3.9379,
  'test_samples_per_second': 2.539,
  'test_steps_per_second': 1.27}}

you can compare it with the performance on regular data: